In [ ]:
# ==================================================
# FEniCSx environment bootstrap (REQUIRED)
# ==================================================

!git clone https://github.com/seoultechpse/fenicsx-colab.git /content/fenicsx-colab || true
!bash /content/fenicsx-colab/bootstrap.sh

# Poisson 2D Complete Example (FEniCSx)

This notebook solves the Poisson equation on the unit square using FEniCSx and MPI.

In [4]:
%%fenicsx --info

UsageError: Cell magic `%%fenicsx` not found.



## Problem description

We solve

$$
-\Delta u = 1 \quad \text{in } \Omega=(0,1)^2,
\qquad
u=0 \quad \text{on } \partial\Omega.
$$

- Finite elements: CG1 (P1)
- Parallel execution using MPI


In [ ]:
%%fenicsx -np 4

import numpy as np
from mpi4py import MPI
from petsc4py import PETSc

from dolfinx import mesh, fem, io
from dolfinx.fem.petsc import LinearProblem
import ufl

# --------------------------------------------------
# Build mesh + solve
# --------------------------------------------------

def solve_poisson(comm, nx=32, ny=32):
    # Mesh & function space
    domain = mesh.create_unit_square(comm, nx, ny)
    V = fem.functionspace(domain, ("Lagrange", 1))

    # Dirichlet BC: u = 0 on boundary
    u_bc = fem.Function(V)
    u_bc.x.array[:] = 0.0

    fdim = domain.topology.dim -1
    boundary_facets = mesh.locate_entities_boundary(
        domain, fdim, lambda x: np.full(x.shape[1], True)
    )
    bc = fem.dirichletbc(
        u_bc,
        fem.locate_dofs_topological(V, fdim, boundary_facets)
    )

    # Variational problem
    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    f = fem.Constant(domain, PETSc.ScalarType(1.0))

    a = ufl.inner(ufl.grad(u), ufl.grad(v)) *ufl.dx
    L = f *v *ufl.dx

    problem = LinearProblem(
        a, L, bcs=[bc],
        petsc_options={"ksp_type": "cg", "pc_type": "hypre"},
        petsc_options_prefix="poisson_",
    )

    uh = problem.solve()

    # Diagnostics
    local_L2 = fem.assemble_scalar(fem.form(uh *uh *ufl.dx))
    global_L2 = comm.allreduce(local_L2, op=MPI.SUM)

    if comm.rank == 0:
        print("✅ Poisson problem solved")
        print("   Number of dofs:", V.dofmap.index_map.size_global)
        print("   MPI size      :", comm.size)
        print("   L2 norm       :", np.sqrt(global_L2))

    return domain, uh

# --------------------------------------------------
# save_xdmf: collective I/O
# --------------------------------------------------

def save_xdmf(comm, domain, uh, filename="poisson.xdmf"):

    with io.XDMFFile(comm, filename, "w") as xdmf:
        xdmf.write_mesh(domain)
        xdmf.write_function(uh)
    if comm.rank == 0:
        print(f"🖼️ Saved {filename}")

# --------------------------------------------------

comm = MPI.COMM_WORLD

domain, uh = solve_poisson(comm)
save_xdmf(comm, domain, uh)

In [ ]:
from google.colab import files

files.download("poisson.xdmf")
files.download("poisson.h5")